# Special Item Number (SIN) Analysis

In [1]:
import pandas as pd
import re

## Getting SIN descriptions

We'll obtain descriptions for SINs from a CSV export of the [GSA eLibrary Schedules and Contracts](http://catalog.data.gov/dataset/gsa-elibrary-schedules-and-contracts) Excel spreadsheet.

In [2]:
with open('../data/eLibrary_Schedule_Contracts.csv', encoding='ascii', errors='ignore') as f:
    rows = pd.read_csv(f)

rows.drop_duplicates(subset=['SPECIAL_ITEM_NUMBER'], inplace=True)

SINS = {}

for i in rows.index:
    sin = rows['SPECIAL_ITEM_NUMBER'][i]
    desc = rows['SPECIALITEM_NUMBER_DESCRIPTION'][i]
    SINS[sin] = desc

Some SINs aren't in that CSV, so we'll add them manually.

In [3]:
# From http://www.gsa.gov/portal/content/245439
SINS['100 01'] = 'Introduction of New Services'
SINS['100 03'] = 'Ancillary Supplies and/or Services'

In [4]:
SIN_DESCRIPTIONS = pd.Series(SINS)
SIN_DESCRIPTIONS.to_frame(name='Description')

,Description
003 01,Smart Buildings Systems Integrator
003 100,"Ancillary Supplies and/or Services, Relating t..."
003 97,Ancillary Repair and Alterations
023 101,Trailers and Attachments
023 LSV,"Low Speed Vehicles, Gas or Electric"
025 101,Miscellaneous Vehicle Accessories
051 200,Accessories
051 400,Ancillary Supplies and/or Services
051 99,Introduction of New Products and Services
096 1N,Passenger (New)


## Parsing SINs in contracts

This part is challenging because the SIN column in our contracts CSV is free-form; examples of how it's filled out include e.g.:

* `874-1, 874-6, 874-7`
* `874-1, 6, 7` (Shorthand indicating the row belongs to SINs 874-1, 874-6, and 874-7)
* `874-1 thru 7`
* `All SINs` (Meaning, I *think*, that the row belongs to all SINs in its schedule)

Parsing this is hard! We're going to attempt to deal with the first few of these, at least.

In [5]:
prices = pd.read_csv('../data/hourly_prices.csv', index_col=False, thousands=',')

In [6]:
def parse_sins(prices):
    idx = {}
    charset = build_charset(SINS.keys())
    sins = SINS.keys()
    sin_map = {}
    for sin in sins:
        sin_map[tuple(sin.split(' '))] = True
    charset_re = re.compile('[' + build_charset(sins) + ']+')
    prefixes = get_sin_prefixes(SINS.keys())
    prefix_map = {}
    for prefix in prefixes:
        prefix_map[prefix] = True

    # TODO: Some SINs in our data are suffixed with strange characters
    # that don't have entries in our SIN descriptions table. However,
    # the version *without* the suffix is in our table, so we'll
    # strip out the suffix if needed. But we need to figure out what
    # those suffixes mean and whether what we're doing is OK.
    weird_suffixes = ['R', 'RC']

    for i in prices['SIN NUMBER'].index:
        val = str(prices['SIN NUMBER'][i])
        parts = charset_re.findall(val)
        last_prefix = None
        sins = []
        for part in parts:
            if last_prefix is not None:
                sin = (last_prefix, part)
                for weird_suffix in weird_suffixes:
                    if sin not in sin_map and part.endswith(weird_suffix):
                        sin = (last_prefix, part[:-len(weird_suffix)])
                        continue
                if sin in sin_map:
                    sins.append(' '.join(sin))
                    continue
            if part in prefix_map:
                last_prefix = part
            elif part.startswith('C') and part[1:] in prefix_map:
                # TODO: Some SINs in our data are prefixed with the letter
                # 'C' and aren't in our SIN descriptions table, but their
                # variant without the 'C' *is* in our table. We're going
                # to strip out the prefix for now, but we should figure
                # out what it means and whether what we're doing is OK.
                last_prefix = part[1:]
        if sins:
            idx[i] = list(set(sins))
    return pd.Series(idx)

def build_charset(vocab):
    charset = []
    for word in vocab:
        charset.extend(list(word))
    charset = set(charset)
    return ''.join(charset.difference([' ', '-']))

def get_sin_prefixes(sins):
    return list(set([sin.split(' ')[0] for sin in sins]))

sins = pd.DataFrame(index=prices.index)
sins['Raw SIN'] = prices['SIN NUMBER'].fillna('')
sins['Parsed SINs'] = parse_sins(prices)
sins['Labor Category'] = prices['Labor Category']

Here's some of the results of our parsing.

In [7]:
sins.head(5)

,Raw SIN,Parsed SINs,Labor Category
0,"874-1, 874-1RC, 874-7, 874-7RC","[874 1, 874 7]",Project Manager
1,874-4 / 874-4RC,[874 4],Program Coordinator
2,382-2,[382 2],"Interpretation Services Class 4: Afrikan,Akan,..."
3,874-4 / 874-4RC,[874 4],Administrative Support
4,"874-1, 874-1RC, 874-7, 874-7RC","[874 1, 874 7]",Associate


However, not everything could be parsed! Here's some examples of what didn't make it.

In [8]:
sins[sins['Parsed SINs'].isnull()].head(5)

,Raw SIN,Parsed SINs,Labor Category
244,541-4G,NaN,CU-12 - Challenge Consulting Hourly Fee
263,All SINs,NaN,Subject Matter Expert SME 1.0
333,All,NaN,Functional Specialist III
487,All,NaN,Functional Specialist II
504,All SINs,NaN,Subject Matter Expert SME 2.0


Note also that even for the SINs that we did parse, we didn't manage to parse *all* the SINs out:

In [9]:
sins[sins['Raw SIN'].str.contains('thru')].head(2)

,Raw SIN,Parsed SINs,Labor Category
210,871-1 thru -6,"[871 1, 871 6]",Subject Matter Expert V
299,C871-1 thru -6,"[871 1, 871 6]",Executive 4


## Counting SINs in contracts

In [10]:
def get_union(series):
    union = set()

    series[series.notnull()].apply(lambda x: union.update(x))
    return list(union)

priced_sins = pd.DataFrame(index=get_union(sins['Parsed SINs']))
priced_sins['Description'] = SIN_DESCRIPTIONS

def get_item_counts(series):
    counts = {}

    def add_counts(items):
        for item in items:
            counts[item] = counts.get(item, 0) + 1

    series[series.notnull()].apply(add_counts)
    return counts

priced_sins['Count'] = pd.Series(get_item_counts(sins['Parsed SINs']))

%matplotlib inline

priced_sins.sort_values(by='Count', ascending=False)

,Description,Count
874 1,Consulting Services,15566
871 3,"System Design, Engineering and Integration",12681
871 1,Strategic Planning for Technology Programs/Act...,12121
871 2,Concept Development and Requirements Analysis,11942
871 6,Acquisition and Life Cycle Management,11650
871 4,Test and Evaluation,11171
871 5,Integrated Logistics Support,10189
874 7,Program and Project Management,9878
874 501,Supply and Value Chain Management,7851
899 1,Environmental Consulting Services,6994


## Bayesian analysis: predicting the SINs of a labor category

Ok, Eric says this isn't actually Bayesian, but I thought it was based on [Hilary Mason's tutorial](http://techbus.safaribooksonline.com/video/programming/machine-learning/9781491958155/an-introduction-to-machine-learning-with-web-data-by-hilary-mason/video222094). Hmmm!

Anyways, this is just an attempt to see if we can figure out what SINs a labor category belongs to given the words in the labor category.

In [11]:
bayesian_sin_words = {}
bayesian_words = {'_total': 0}

for sin in priced_sins.index:
    bayesian_sin_words[sin] = {'_total': 0}

for i in sins[sins['Parsed SINs'].notnull()].index:
    parsed_sins = sins['Parsed SINs'][i]
    # TODO: Consider using a stemmer here?
    words = sins['Labor Category'][i].lower().split()
    for word in words:
        if not word:
            continue
        bayesian_words[word] = bayesian_words.get(word, 0) + 1
        bayesian_words['_total'] += 1
        for sin in parsed_sins:
            bayesian_sin_words[sin][word] = bayesian_sin_words[sin].get(word, 0) + 1
            bayesian_sin_words[sin]['_total'] += 1

In [12]:
def get_word_probability(word, sin):
    '''Given a word, what is the probability it is associated with the given SIN?'''
    
    prob_of_word_given_sin = float(bayesian_sin_words[sin].get(word, 0)) / bayesian_sin_words[sin]['_total']
    prob_of_sin = float(priced_sins['Count'][sin]) / prices.shape[0]
    prob_of_word = float(bayesian_words.get(word, 0)) / bayesian_words['_total']
    if prob_of_word == 0:
        return 0
    return prob_of_word_given_sin * prob_of_sin / prob_of_word

def get_probable_sins(labor_category):
    probs = {}
    words = labor_category.lower().split()
    for sin in priced_sins.index:
        prob = 1
        for word in words:
            prob *= get_word_probability(word, sin)
        probs[sin] = prob
    df = pd.DataFrame(index=priced_sins.index)
    df['Description'] = SIN_DESCRIPTIONS
    df['Probability'] = pd.Series(probs)
    return df.sort_values(by='Probability', ascending=False)

get_probable_sins('creative director').head(10)

,Description,Probability
541 4B,Video/Film Production,0.128434
541 1,Advertising Services,0.096519
541 5,Integrated Marketing Services,0.061000
541 4F,Commercial Art and Graphic Design Services (Sm...,0.051847
541 3,Web Based Marketing Services (Small Business S...,0.050708
541 4A,Market Research and Analysis,0.044692
874 1,Consulting Services,0.041790
541 2,Public Relations Services,0.034432
541 4D,"Conference, Events and Tradeshow Planning Serv...",0.015071
541 4C,Exhibit Design and Implementation Services,0.013104
